# Generate Data set 

In [3]:
!python ../main_generate_dataset.py


Bad key "text.kerning_factor" on line 4 in
/Users/seongjungkim/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
100%|████████████████████████████████████| 11000/11000 [01:09<00:00, 158.50it/s]
consume_time : 85.67403197288513
transaction units : 128.39363044663727
Finish generate detection dataset!


# Training Model

In [ ]:
import sys
sys.path.append('../')
from model import simple_detection_netowrk
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from loss import detection_loss, ssd_loss

# load dataset
train_xs = np.load('../datasets/true_images.npy')
train_ys = np.load('../datasets/true_labels.npy')

input_shape = train_xs.shape[1:]
n_classes = 11
train_ys_cls = train_ys[..., 4:]
train_ys_cls = np.where(train_ys_cls == -1, 10, train_ys_cls)
train_ys_cls = to_categorical(train_ys_cls, num_classes=11)
train_ys = np.concatenate([train_ys[..., :4], train_ys_cls], axis=-1)

# Generate detection SSD model
n_boxes = 5
inputs, predictions = simple_detection_netowrk(input_shape, n_boxes, n_classes)

model = Model(inputs, predictions)
model.compile('adam', loss=ssd_loss)

pred_ = model.predict(train_xs)
model.fit(x=train_xs, y=train_ys)
